In [2]:
import pycaret.classification as pc
import pandas as pd
from sklearn.metrics import log_loss
import mlflow
import matplotlib.pyplot as plt

# get env name
import os
import sys
env_name = os.path.basename(os.path.dirname(sys.executable))
#print(env_name)

X_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_test.parquet")
y_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_test.parquet")

X_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_train.parquet")
y_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_train.parquet")
data_train = pd.concat([X_train, y_train], axis=1)

mlflow.set_tracking_uri('http://localhost:5000/')

EXPERIMENT_NAME = "kobe"
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

tags = {
        "Projeto": "22E1_3_Kobe",
        "team": "Daniel S. U. Tamashiro",
        "dataset": "Kobe Bryant Shot Selection"
       }


RUN_NAME= env_name
with mlflow.start_run(experiment_id=experiment, run_name=RUN_NAME) as run:
#with mlflow.start_run():


    clf = pc.setup(data = data_train, target = "shot_made_flag")
    lr = pc.create_model('lr')
    tuned_lr = pc.tune_model(lr)

    pc.plot_model(tuned_lr, plot='auc')
    temp_name = "auc.png"
    plt.savefig(temp_name)
    mlflow.log_artifact(temp_name, "Curva ROC")

    y_pred = lr.predict_proba(X_test)
    log_loss_score = log_loss(y_test, y_pred)
    print('Log loss score: {:.4f}'.format(log_loss_score))

    #Track tags
    mlflow.set_tags(tags)

    # Track parameters
    mlflow.log_param("target", "shot_made_flag")
    mlflow.log_param("model", "lr")

    # Track metrics
    mlflow.log_metric("log_loss", log_loss_score)

    # Track model
    #mlflow.sklearn.log_model(pipe, "model")
    
    #Registro do modelo
    mlflow.log_artifact(local_path='./model.ipynb',artifact_path='code')
    


mlflow.end_run()


KeyboardInterrupt: 

In [6]:
import pycaret.classification as pc
import pandas as pd
from sklearn.metrics import log_loss
import mlflow
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

X_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_test.parquet")
y_test = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_test.parquet")

X_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/X_train.parquet")
y_train = pd.read_parquet("C:/Users/shtsu/Codigos/23E1_3/projeto/data/03_primary/y_train.parquet")
data_train = pd.concat([X_train, y_train], axis=1)

mlflow.set_tracking_uri('http://localhost:5000/')

EXPERIMENT_NAME = "kobe"
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

tags = {
        "Projeto": "22E1_3_Kobe",
        "team": "Daniel S. U. Tamashiro",
        "dataset": "Kobe Bryant Shot Selection"
       }

with mlflow.start_run(experiment_id=experiment, run_name="Treinamento") as run:
    # Log pipeline parameters to MLflow
    mlflow.log_param("scaler", "StandardScaler")
    mlflow.log_param("classifier", "LogisticRegression")

    clf = pc.create_model('lr', penalty='l2')
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', clf)
    ])
    tuned_pipe = pc.tune_model(pipe)

    pc.plot_model(tuned_pipe, plot='auc')
    temp_name = "auc.png"
    plt.savefig(temp_name)
    mlflow.log_artifact(temp_name, "Curva ROC")

    y_pred = tuned_pipe.predict_proba(X_test)
    log_loss_score = log_loss(y_test, y_pred)
    print('Log loss score: {:.4f}'.format(log_loss_score))

    # Track tags
    mlflow.set_tags(tags)

    # Track parameters
    mlflow.log_param("target", "shot_made_flag")
    mlflow.log_param("model", "lr")

    # Track metrics
    mlflow.log_metric("log_loss", log_loss_score)

    # Track model
    mlflow.sklearn.log_model(tuned_pipe, "model")
    
    #Registro do modelo
    mlflow.log_artifact(local_path='./model.ipynb',artifact_path='code')
    

mlflow.end_run()



,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:27:41
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


Processing:   0%|          | 0/4 [00:00<?, ?it/s]